### Function definition & File input

In [1]:
import math
import os
import numpy
import matplotlib.pyplot as plt
import numpy as np
#######################################################################
data_t = None
data_x = None
train_t = []
valid_t = []
train_x = []
valid_x = []
D = 7
feature_name = ['GRE_score','TOFEL_score','University_rating','SOP','LOR','CGPA','Research']
# Data t: Chance_of_Admit 
# Data x: GRE_score,TOFEL_score,University_rating,SOP,LOR ,CGPA,Research
#######################################################################
def read_file():
    global data_t, data_x
    file_t = open("./dataset/data_T.csv", mode = 'r')
    file_x = open("./dataset/data_X.csv", mode = 'r')
    data_t = file_t.readlines()
    data_x = file_x.readlines()
    # process data T
    for i in range(500):
        data_t[i] = data_t[i].split(',')
        data_t[i] = data_t[i][1].split('\n')[0]
        data_t[i] = float(data_t[i])
    # process data X
    for i in range(500):
        data_x[i] = data_x[i].split(',')
        data_x[i][D] = data_x[i][7].split('\n')[0]
        data_x[i] = data_x[i][1:]
        for j in range(7):
            data_x[i][j] = float(data_x[i][j])
        
    file_t.close()
    file_t.close()
def preprocess():
    m = []
    s = []
    N = len(data_x)
    for j in range(6):
        tmp_list = []
        for i in range(N):
            tmp_list.append(data_x[i][j])
        m.append(np.mean(tmp_list))
        s.append(math.sqrt(np.var(tmp_list)))
    for i in range(len(data_x)):
        for j in range(6):
            data_x[i][j] = (data_x[i][j] - m[j])/s[j]
def split_dataset():
    global train_t, train_x, test_t, test_x
    for i in range(400):
        train_t.append(data_t[i])
        train_x.append(data_x[i])
    for i in range(400,500):
        valid_t.append(data_t[i])
        valid_x.append(data_x[i])
def error_func(x,t,w,N,M):
    err = 0
    for i in range(N):
        err += (poly_func(x[i],w,M) - t[i])**2
    err = err/(2*N)
    return err
def poly_func(x,w,M):
    val = 0.0
    # w is a 1D list
    if(M==1): 
        val += w[0]
        for i in range(D):
            val += w[i+1]*x[i]
    # M = 2
    # w is a 1D list
    elif(M==2):
        val += w[0]
        for i in range(1,D+1):
            val += w[i]*x[i-1]
        cnt = 8
        for i in range(7):
            for j in range(7):
                val += w[cnt]*x[i]*x[j]
                cnt +=1
    # Poly
    else: 
        val += w[0]
        for i in range(1,D+1):
            val += w[i]*x[i-1]
        cnt = 8
        for i in range(0,7):
            for j in range(i,7):
                val += w[cnt]*x[i]*x[j]
                cnt +=1
    return val
def update_weights_1(x,t,w,learning_rate):
    w_derive = [0]*(D+1)
    N = len(x)
    # find derive of w[0]
    for i in range(N):
        sum = 0
        sum += w[0]
        for j in range(D):
            sum += w[j+1]*x[i][j]
        sum -= t[i]
        w_derive[0] += sum
    # find derive of w[1]~w[7]
    for cnt in range(1,D+1):
        for i in range(N):
            sum = 0
            sum += w[0]
            for j in range(D):
                sum += w[j+1]*x[i][j]
            sum -= t[i]
            sum *= x[i][cnt-1]
            w_derive[cnt] += sum

    for i in range(D+1):
        w[i] -= (w_derive[i]/float(N)) * learning_rate
    
    return w
def update_weights_2(x,t,w,learning_rate):
    w_derive = [0]*36
    N = len(x)
    # find derive of w[0]
    for i in range(N):
        sum = 0
        sum += w[0]
        for j in range(7):
            sum += w[j+1]*x[i][j]
        for j in range(7):
            for k in range(7):
                sum += w[(j+1)*7+(k+1)]*x[i][j]*x[i][k]
        sum -= t[i]
        w_derive[0] += sum
    # find derive of w[1]~w[7]
    for cnt in range(1,8):
        for i in range(N):
            sum = 0
            sum += w[0]
            for j in range(7):
                sum += w[j+1]*x[i][j]
            for j in range(7):
                for k in range(7):
                    sum += w[(j+1)*7+(k+1)]*x[i][j]*x[i][k]
            sum -= t[i]
            sum *= x[i][cnt-1]
            w_derive[cnt] += sum
    # find derive of w[8]~w[56]
    for cnt in range(8,36):
        for i in range(N):
            sum = 0
            sum += w[0]
            for j in range(7):
                sum += w[j+1]*x[i][j]
            for j in range(7):
                for k in range(7):
                    sum += w[(j+1)*7+(k+1)]*x[i][j]*x[i][k]
            sum -= t[i]
            
            sum *= x[i][((cnt-1)//7)-1]*x[i][(cnt-1)%7]
            w_derive[cnt] += sum
    # complete w[0] ~ w[56]
    for i in range(36):
         w[i] -= (w_derive[i]/float(N)) * learning_rate
    return w
def train_1(train_x,train_t,learning_rate,iter_num):
    w = [0]*(D+1)
    w[0] = 1
    err_list = []
    N = len(train_x)
    for i in range(iter_num):
        w = update_weights_1(train_x,train_t, w, learning_rate)
        
        tmp_err = error_func(train_x,train_t,w,N,1)
        err_list.append(tmp_err)
        #print(tmp_err)
    return w
def train_2(train_x,train_t,learning_rate,iter_num):
    w = [0]*36
    w[0] = 1
    
    err_list = []
    N = len(train_x)
    for i in range(iter_num):
        w = update_weights_2(train_x,train_t, w, learning_rate)
        
        tmp_err = error_func(train_x,train_t,w,N,2)
        err_list.append(tmp_err)
    return w
def calculate_RMS(x,t,w,M):
    N = len(x)
    RMS = 0.0
    #print(w)
    for i in range(N):
        RMS += (poly_func(x[i],w,M) - t[i])**2
    RMS = math.sqrt(RMS/float(N))
    return RMS
def select_feature(f_num):
    t_x = [0]*400
    v_x = [0]*100
    for i in range(400):
        t_x[i] = []
        for n in f_num:
            t_x[i].append(train_x[i][n])
    for i in range(100):
        v_x[i] = []
        for n in f_num:
            v_x[i].append(valid_x[i][n])
    return t_x,v_x
def apply_poly(train_x,train_t,l):
    x = np.array(train_x,dtype='float')
    t = np.array(train_t,dtype='float').reshape(400,1)
    one = np.array([[1]*400],dtype='float')
    # add coefficient for w0
    x = np.insert(x,0,values=one, axis = 1)
    # x = x1 +...+x7 +x1x1 + x1x2 + ....+ x7x7 (total 36 items)
    for i in range(7):
        for j in range(i,7):
            tmp = np.array([1]*400,dtype='float').reshape(400,1)
            for c in range(400):
                tmp[c] = train_x[c][i]*train_x[c][j]    
            x = np.append(x,values=tmp, axis = 1)
    if(l == None):
        w = np.dot(np.dot(np.linalg.pinv(np.dot(x.transpose(),x)),x.transpose()),t)
    else:
        w = np.dot(np.dot(np.linalg.pinv(np.dot(x.transpose(),x)++np.eye(36)*l),x.transpose()),t)
    return w
def least_square_sol_1(train_x,train_t):
    x = np.array(train_x,dtype='float')
    t = np.array(train_t,dtype='float').reshape(400,1)
    one = np.array([[1]*400],dtype='float')
    # add coefficient for w0
    x = np.insert(x,0,values=one, axis = 1)
    #
    w = np.dot(np.dot(np.linalg.inv(np.dot(x.transpose(),x)),x.transpose()),t)
    return w
def reg_least_square_sol_1(train_x,train_t,l):
    x = np.array(train_x,dtype='float')
    t = np.array(train_t,dtype='float').reshape(400,1)
    one = np.array([[1]*400],dtype='float')
    # add coefficient for w0
    x = np.insert(x,0,values=one, axis = 1)
    #
    w = np.dot(np.dot(np.linalg.inv(np.dot(x.transpose(),x)+np.eye(8)*l),x.transpose()),t)
    return w
def least_square_sol_2(train_x,train_t):
    x = np.array(train_x,dtype='float')
    t = np.array(train_t,dtype='float').reshape(400,1)
    one = np.array([[1]*400],dtype='float')
    # add coefficient for w0
    x = np.insert(x,0,values=one, axis = 1)
    # 
    for i in range(7):
        for j in range(7):
            tmp = np.array([1]*400,dtype='float').reshape(400,1)
            for c in range(400):
                tmp[c] = train_x[c][i]*train_x[c][j]    
            x = np.append(x,values=tmp, axis = 1)
    w = np.dot(np.dot(np.linalg.pinv(np.dot(x.transpose(),x)),x.transpose()),t)
    return w
def reg_least_square_sol_2(train_x,train_t,l):
    x = np.array(train_x,dtype='float')
    t = np.array(train_t,dtype='float').reshape(400,1)
    one = np.array([[1]*400],dtype='float')
    # add coefficient for w0
    x = np.insert(x,0,values=one, axis = 1)
    # 
    for i in range(7):
        for j in range(7):
            tmp = np.array([1]*400,dtype='float').reshape(400,1)
            for c in range(400):
                tmp[c] = train_x[c][i]*train_x[c][j]    
            x = np.append(x,values=tmp, axis = 1)
    w = np.dot(np.dot(np.linalg.pinv(np.dot(x.transpose(),x)+np.eye(57)*l),x.transpose()),t)
    return w
############################################################
read_file()
#preprocess()
split_dataset()

### 2-1 Feature select

In [2]:
# M = 1
w_1 = least_square_sol_1(train_x,train_t)
RMS_train_1 = calculate_RMS(train_x,train_t,w_1,1)
RMS_valid_1 = calculate_RMS(valid_x,valid_t,w_1,1)
print("RMS error of training set (M=1) = ", RMS_train_1)
print("RMS error of validation set (M=1) = ", RMS_valid_1)
# M = 2
w = least_square_sol_2(train_x,train_t)
RMS_train = calculate_RMS(train_x,train_t,w,2)
RMS_valid = calculate_RMS(valid_x,valid_t,w,2)
print("\nRMS error of training set (M=2) = ", RMS_train)
print("RMS error of validation set (M=2) = ", RMS_valid)
# Find most contributive feature
contributive_feature_num = -1
val = 0
for i in range(7):
    if(w_1[i+1]**2 > val):
        contributive_feature_num = i
        val = w_1[i+1]**2
print("\nMost contributive feature = ", feature_name[contributive_feature_num])

RMS error of training set (M=1) =  0.06314185017282051
RMS error of validation set (M=1) =  0.04293220746251964

RMS error of training set (M=2) =  0.06015696005703246
RMS error of validation set (M=2) =  0.048938858565618436

Most contributive feature =  CGPA


### 2-2(b) Introduce basis function to previous model

In [3]:
# Polynomial basis function
w = apply_poly(train_x,train_t,None)
RMS_train = calculate_RMS(train_x,train_t,w,3)
RMS_valid = calculate_RMS(valid_x,valid_t,w,3)
print("\nRMS error of training set (Poly) = ", RMS_train)
print("RMS error of validation set (Poly) = ", RMS_valid)


RMS error of training set (Poly) =  0.06015694892729082
RMS error of validation set (Poly) =  0.04894526470755938


### 2-2(C) N-fold cross validation

In [4]:
# K-fold cross-validation
#
v_x = []
v_x.append(data_x[0:100])
v_x.append(data_x[100:200])
v_x.append(data_x[200:300])
v_x.append(data_x[300:400])
v_x.append(data_x[400:500])
v_t = []
v_t.append(data_t[0:100])
v_t.append(data_t[100:200])
v_t.append(data_t[200:300])
v_t.append(data_t[300:400])
v_t.append(data_t[400:500])
#
t_t = []
t_t.append(v_t[1])
t_t[0] = np.append(t_t[0],v_t[2],0)
t_t[0] = np.append(t_t[0],v_t[3],0)
t_t[0] = np.append(t_t[0],v_t[4],0)

t_t.append(v_t[0])
t_t[1] = np.append(t_t[1],v_t[2],0)
t_t[1] = np.append(t_t[1],v_t[3],0)
t_t[1] = np.append(t_t[1],v_t[4],0)

t_t.append(v_t[0])
t_t[2] = np.append(t_t[2],v_t[1],0)
t_t[2] = np.append(t_t[2],v_t[3],0)
t_t[2] = np.append(t_t[2],v_t[4],0)

t_t.append(v_t[0])
t_t[3] = np.append(t_t[3],v_t[1],0)
t_t[3] = np.append(t_t[3],v_t[2],0)
t_t[3] = np.append(t_t[3],v_t[4],0)

t_t.append(v_t[0])
t_t[4] = np.append(t_t[4],v_t[1],0)
t_t[4] = np.append(t_t[4],v_t[2],0)
t_t[4] = np.append(t_t[4],v_t[3],0)
#
t_x = []
t_x.append(v_x[1])
t_x[0] = np.append(t_x[0],v_x[2],0)
t_x[0] = np.append(t_x[0],v_x[3],0)
t_x[0] = np.append(t_x[0],v_x[4],0)

t_x.append(v_x[0])
t_x[1] = np.append(t_x[1],v_x[2],0)
t_x[1] = np.append(t_x[1],v_x[3],0)
t_x[1] = np.append(t_x[1],v_x[4],0)

t_x.append(v_x[0])
t_x[2] = np.append(t_x[2],v_x[1],0)
t_x[2] = np.append(t_x[2],v_x[3],0)
t_x[2] = np.append(t_x[2],v_x[4],0)

t_x.append(v_x[0])
t_x[3] = np.append(t_x[3],v_x[1],0)
t_x[3] = np.append(t_x[3],v_x[2],0)
t_x[3] = np.append(t_x[3],v_x[4],0)

t_x.append(v_x[0])
t_x[4] = np.append(t_x[4],v_x[1],0)
t_x[4] = np.append(t_x[4],v_x[2],0)
t_x[4] = np.append(t_x[4],v_x[3],0)
# N-fold
RMS_train_m = 0
RMS_valid_m = 0
for i in range(5):
    w = least_square_sol_1(t_x[i],t_t[i])
    RMS_train_m += calculate_RMS(t_x[i],t_t[i],w,1)
    RMS_valid_m += calculate_RMS(v_x[i],v_t[i],w,1)

print("\nRMS error of training set(K-fold, M = 1) = ", RMS_train_m/5.0)
print("RMS error of valid set(K-fold, M = 1) = ", RMS_valid_m/5.0)
RMS_train_m = 0
RMS_valid_m = 0
for i in range(5):
    w = least_square_sol_2(t_x[i],t_t[i])
    RMS_train_m += calculate_RMS(t_x[i],t_t[i],w,2)
    RMS_valid_m += calculate_RMS(v_x[i],v_t[i],w,2)
print("\nRMS error of training set(K-fold, M = 2) = ", RMS_train_m/5.0)
print("RMS error of valid set(K-fold, M = 2) = ", RMS_valid_m/5.0)


RMS error of training set(K-fold, M = 1) =  0.058796957934396335
RMS error of valid set(K-fold, M = 1) =  0.05946970564727

RMS error of training set(K-fold, M = 2) =  0.05579987637150844
RMS error of valid set(K-fold, M = 2) =  0.06395489510640454


### 2-3(b) Use MAP to retest the model

In [5]:
# Poly MAP
w = apply_poly(train_x,train_t,1)
RMS_train = calculate_RMS(train_x,train_t,w,3)
RMS_valid = calculate_RMS(valid_x,valid_t,w,3)
print("\nRMS error of training set (Poly, lambda = 1) = ", RMS_train)
print("RMS error of validation set (Poly, lambda = 1) = ", RMS_valid)
w = apply_poly(train_x,train_t,100)
RMS_train = calculate_RMS(train_x,train_t,w,3)
RMS_valid = calculate_RMS(valid_x,valid_t,w,3)
print("\nRMS error of training set (Poly, lambda = 100) = ", RMS_train)
print("RMS error of validation set (Poly, lambda = 100) = ", RMS_valid)
w = apply_poly(train_x,train_t,10000)
RMS_train = calculate_RMS(train_x,train_t,w,3)
RMS_valid = calculate_RMS(valid_x,valid_t,w,3)
print("\nRMS error of training set (Poly, lambda = 10000) = ", RMS_train)
print("RMS error of validation set (Poly, lambda = 10000) = ", RMS_valid)
# N-fold MAP
RMS_train_m = 0
RMS_valid_m = 0
for i in range(5):
    w = reg_least_square_sol_1(t_x[i],t_t[i],1)
    RMS_train_m += calculate_RMS(t_x[i],t_t[i],w,1)
    RMS_valid_m += calculate_RMS(v_x[i],v_t[i],w,1)

print("\nRMS error of training set(K-fold, MAP, lambda = 1, M = 1) = ", RMS_train_m/5.0)
print("RMS error of valid set(K-fold, MAP, lambda = 1, M = 1) = ", RMS_valid_m/5.0)
RMS_train_m = 0
RMS_valid_m = 0
for i in range(5):
    w = reg_least_square_sol_1(t_x[i],t_t[i],100)
    RMS_train_m += calculate_RMS(t_x[i],t_t[i],w,1)
    RMS_valid_m += calculate_RMS(v_x[i],v_t[i],w,1)

print("\nRMS error of training set(K-fold, MAP, lambda = 100, M = 1) = ", RMS_train_m/5.0)
print("RMS error of valid set(K-fold, MAP, lambda = 100, M = 1) = ", RMS_valid_m/5.0)
RMS_train_m = 0
RMS_valid_m = 0
for i in range(5):
    w = reg_least_square_sol_1(t_x[i],t_t[i],10000)
    RMS_train_m += calculate_RMS(t_x[i],t_t[i],w,1)
    RMS_valid_m += calculate_RMS(v_x[i],v_t[i],w,1)

print("\nRMS error of training set(K-fold, MAP, lambda = 10000, M = 1) = ", RMS_train_m/5.0)
print("RMS error of valid set(K-fold, MAP, lambda = 10000, M = 1) = ", RMS_valid_m/5.0)



RMS error of training set (Poly, lambda = 1) =  0.06044961263428089
RMS error of validation set (Poly, lambda = 1) =  0.04626932554473747

RMS error of training set (Poly, lambda = 100) =  0.06164013757778419
RMS error of validation set (Poly, lambda = 100) =  0.04556290207786235

RMS error of training set (Poly, lambda = 10000) =  0.06337523725922141
RMS error of validation set (Poly, lambda = 10000) =  0.04347689084900574

RMS error of training set(K-fold, MAP, lambda = 1, M = 1) =  0.06422023057691623
RMS error of valid set(K-fold, MAP, lambda = 1, M = 1) =  0.06524071081489292

RMS error of training set(K-fold, MAP, lambda = 100, M = 1) =  0.0739798744796576
RMS error of valid set(K-fold, MAP, lambda = 100, M = 1) =  0.07603777114413872

RMS error of training set(K-fold, MAP, lambda = 10000, M = 1) =  0.1081575913783254
RMS error of valid set(K-fold, MAP, lambda = 10000, M = 1) =  0.10805782204611139
